# Baseline: Finetune Only The Last 2 Layers (Without LoRA)

In [ ]:
# pip install transformers datasets lightning watermark

In [1]:
%load_ext watermark
%watermark -p torch,transformers,datasets,lightning

/home/a14-hliu/miniconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch       : 2.5.1
transformers: 4.46.2
datasets    : 3.1.0
lightning   : 2.5.1.post0



# 1 Loading the dataset into DataFrames

In [ ]:
# Standard imports
import os
from datasets import load_dataset

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

import pandas as pd
import torch

# Custom utilities for downloading and preparing the dataset
from local_dataset_utilities import download_dataset, load_dataset_into_to_dataframe, partition_dataset
from local_dataset_utilities import IMDBDataset

In [ ]:
# Ensure GPU is available before proceeding
if not torch.cuda.is_available():
    print("Please switch to a GPU machine before running this notebook.")

In [ ]:
# Determine whether to download dataset or not based on existence of files
files = ("test.csv", "train.csv", "val.csv")
download = True

for f in files:
    if os.path.exists(os.path.join("data", f)):
        download = False

# if download is True:
#     download_dataset()
#     df = load_dataset_into_to_dataframe()
#     partition_dataset(df)

In [ ]:
# Load already-prepared train/val/test CSVs
df_train = pd.read_csv(os.path.join("data", "train.csv"))
df_val = pd.read_csv(os.path.join("data", "val.csv"))
df_test = pd.read_csv(os.path.join("data", "test.csv"))

# 2 Tokenization and Numericalization

**Load the dataset via `load_dataset`**

In [ ]:
# Load the CSVs into Hugging Face's DatasetDict format
imdb_dataset = load_dataset(
    "csv",
    data_files={
        "train": os.path.join("data", "train.csv"),
        "validation": os.path.join("data", "val.csv"),
        "test": os.path.join("data", "test.csv"),
    },
)

print(imdb_dataset)

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 19
    })
    validation: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 14
    })
    test: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 10
    })
})


**Tokenize the dataset**

In [ ]:
# Load tokenizer from Hugging Face for distilBERT
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


In [ ]:
# Tokenize the dataset for BERT-style input 
def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

In [ ]:
# Apply tokenizer to all splits
imdb_tokenized = imdb_dataset.map(tokenize_text, batched=True, batch_size=None)

Map: 100%|██████████| 10/10 [00:00<00:00, 976.35 examples/s]


In [ ]:
# Free memory by deleting original dataset
del imdb_dataset

In [ ]:
# Set tensor format for PyTorch and specify usable columns
imdb_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# Disable parallel tokenizer warnings
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 3 Set Up DataLoaders

In [ ]:
# Create custom PyTorch Dataset wrapper for DataLoader
from torch.utils.data import DataLoader, Dataset


class IMDBDataset(Dataset):
    def __init__(self, dataset_dict, partition_key="train"):
        self.partition = dataset_dict[partition_key]

    def __getitem__(self, index):
        return self.partition[index]

    def __len__(self):
        return self.partition.num_rows

In [ ]:
# Create datasets for each split
train_dataset = IMDBDataset(imdb_tokenized, partition_key="train")
val_dataset = IMDBDataset(imdb_tokenized, partition_key="validation")
test_dataset = IMDBDataset(imdb_tokenized, partition_key="test")

# Create dataloaders for training, validation, and testing
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=12,
    shuffle=True, 
    num_workers=4
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=12,
    num_workers=4
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=12,
    num_workers=4
)

# 4 Initializing DistilBERT

In [ ]:
# Load pre-trained BERT model for sequence classification
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Freeze all layers**

In [ ]:
# Utility to count trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print("Total number of trainable parameters:", count_parameters(model))

Total number of trainable parameters: 66955010


In [ ]:
# Freeze all parameters initially

for param in model.parameters():
    param.requires_grad = False

**Unfreeze last layer**

In [ ]:
model  # show model architecture

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
# Unfreeze only the classifier layers (i.e., last few layers)

for param in model.pre_classifier.parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True

In [20]:
print("Total number of trainable parameters:", count_parameters(model))

Total number of trainable parameters: 592130


# 5 Finetuning

**Wrap in LightningModule for Training**

In [ ]:
# Wrap model using PyTorch Lightning module
from local_model_utilities import CustomLightningModule

lightning_model = CustomLightningModule(model)

In [ ]:
# Define callbacks: save best model by validation accuracy

from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger


callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_acc"
    )   # Save only the best checkpoint
]
logger = CSVLogger(save_dir="logs/", name="distilbert_lastlayer")

In [ ]:
# Setup Lightning Trainer

trainer = L.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    # accelerator="gpu", # Uncomment if running on GPU manually
    precision="16-mixed",
    devices=1,
    logger=logger,
    log_every_n_steps=10,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
# Train the model and track elapsed time

# import time
# start = time.time()

# trainer.fit(model=lightning_model,
#             train_dataloaders=train_loader,
#             val_dataloaders=val_loader)

# end = time.time()
# elapsed = end - start
# print(f"Time elapsed {elapsed/60:.2f} min")

In [ ]:
# Load the best saved model checkpoint

saved_model_path="logs/distilbert_lastlayer/version_0/checkpoints/epoch=2-step=8751.ckpt" # trained in the full-data
# train_model_path="logs/distilbert_lastlayer/version_1/checkpoints/epoch=0-step=2.ckpt"

In [ ]:
# Evaluate model on each dataset split

train_acc = trainer.test(lightning_model, dataloaders=train_loader, ckpt_path=saved_model_path, verbose=False)
val_acc = trainer.test(lightning_model, dataloaders=val_loader, ckpt_path=saved_model_path, verbose=False)
test_acc = trainer.test(lightning_model, dataloaders=test_loader, ckpt_path=saved_model_path, verbose=False)

You are using a CUDA device ('NVIDIA H100 PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at logs/distilbert_lastlayer/version_0/checkpoints/epoch=2-step=8751.ckpt
/home/a14-hliu/miniconda3/envs/py311/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:362: The dirpath has changed from 'logs/my-model/version_0/checkpoints' to 'logs/distilbert_lastlayer/version_2/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at logs/distilbert_lastlayer/version_0

Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  3.11it/s]

Restoring states from the checkpoint path at logs/distilbert_lastlayer/version_0/checkpoints/epoch=2-step=8751.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at logs/distilbert_lastlayer/version_0/checkpoints/epoch=2-step=8751.ckpt


Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 90.28it/s]


Restoring states from the checkpoint path at logs/distilbert_lastlayer/version_0/checkpoints/epoch=2-step=8751.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at logs/distilbert_lastlayer/version_0/checkpoints/epoch=2-step=8751.ckpt


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 77.11it/s]


In [ ]:
# Print final evaluation results

print(f"Train acc: {train_acc[0]['accuracy']*100:2.2f}%")
print(f"Val acc:   {val_acc[0]['accuracy']*100:2.2f}%")
print(f"Test acc:  {test_acc[0]['accuracy']*100:2.2f}%")

Train acc: 84.21%
Val acc:   85.71%
Test acc:  80.00%


In [ ]:
# Optional cleanup of model checkpoints and logs

# import shutil

# # Cleanup checkpoint files as we don't need them later
# log_dir = f"logs/my-model"
# if os.path.exists(log_dir):
#     shutil.rmtree(log_dir)